Таблица представлена следующими полями:
1) key
2) id
3) phone
4) mail

Тестовые данные:

1;12345;89997776655; test@mail.ru
2;54321;87778885566; two@mail.ru
3;98765; 87776664577;three@mail
4; 66678;87778885566; four@mail.ru
5; 34567; 84547895566; four@mail.ru
6; 34567; 89087545678; five@mail.ru

На основании заданного поля (это может быть id, phone, mail) получить все "связанные данные"

Например:
если задать поиск по условию phone = 87778885566;

Результат должен быть следующим:

2;54321;87778885566; two@mail.ru
4; 66678;87778885566; four@mail.ru
5; 34567; 84547895566; four@mail.ru
6; 34567; 89087545678; five@mail.ru


In [1]:
import numpy as np
import pandas as pd
import random
import string

In [2]:
# Сгенерируем 10'000 данных

all_keys = np.arange(1, 10001)
all_id = np.arange(12345, 98765)
all_phones = np.arange(87778880000, 87778885567)    
n = 10000 

In [3]:
# Рандомайзер для email
def random_char(char_num):
       return ''.join(random.choice(string.ascii_letters) for _ in range(char_num))
n=0
all_mails=[]
while n < 10000:
    all_mails.append(random_char(4)+"@gmail.com")
    n+=1

In [4]:
ids = np.random.choice(all_id, n)
phones = np.random.choice(all_phones, n)
all_mails = np.random.choice(all_mails, n)

In [5]:
# И запишем данные в датафрейм

big_data = pd.DataFrame({'key': all_keys,
                         'id': ids,
                         'phone': phones,
                         'mail': all_mails})

In [6]:
big_data.head()

,key,id,phone,mail
0,1,57819,87778884800,szOQ@gmail.com
1,2,83851,87778885320,zSbY@gmail.com
2,3,18762,87778882516,eTfi@gmail.com
3,4,23242,87778881623,rSal@gmail.com
4,5,59777,87778885512,mbah@gmail.com


In [7]:
def data_to_search(data):
    print('Введите запрос формата <поле> = <значение>. Пример: phone = 87778885566\n')
    search = input('Что ищем? ')
    
    # Чистим запрос. Strip - удаляем пробелы в начале и конце, если они есть
    # Split - делим по пробелам, первое слово - колонка, последнее - значение
    search = search.strip().split(' ')
    
    column_raw = search[0]
    value = search[-1] # Тк берем 1 элемент с конца, будет работать и для запросов без '=', например 'phone 87778885568'

    # Уберем кавычки или скобки, если пользователь их ввел    
    column = ''.join(filter(str.isalpha, column_raw))
    
    if column not in ['key', 'id', 'phone', 'mail']:
        print(f'''\nПоле {column} не найдено. Поиск доступен для полей: key, id, phone, mail''')
        return None
    
    # Обработаем исключение, если пользователь искал текст (имейл) - код не упадет из-за ошибки преобразования текста в int
    if column != 'mail':
        try:
            value = int(value)
        except ValueError:
            print(f'''\nДля поля {column} допустимы только числа.
                  \nЗначение {value} не распознано.''')
            return None
    
    result = data.query("{0} == @value".format(column))
    print(f'\nДля поля {column} по запросу {value} найдено {result.shape[0]} записей')
    return result

In [8]:
def write_data(investigation):
    if investigation is not None:   
        print(investigation) # Добавить head если данных ожидается МНОГО

        # Запишем данные в csv. По ТЗ разделитель ';'. Дропнем индексы и хедеры
        data = investigation.to_csv('i_found.csv', sep=';', index=False, header=False)

    else:
        print('Данные не были записаны. Проверьте запрос')

In [9]:
# Вызовем функцию и запишем результат в переменную
# Тестовые запросы:
# phone = 87778885568
# mail = test@mail.ru

big_investigation = data_to_search(big_data)

Введите запрос формата <поле> = <значение>. Пример: phone = 87778885566

Что ищем? phone = 87778885566

Для поля phone по запросу 87778885566 найдено 1 записей


In [10]:
# При желании можно поработать с вводом пользователя на вопрос опечаток,
# но это выходит за рамки задания

# Поиск связанных данных

In [11]:
# Функция объединения df по одной колонке (column)

def merge_column(big_data, current_data, column):
    output = big_data.merge(current_data, on=column, how='inner', suffixes=('', '_y'))
    output.drop(output.filter(regex='_y$').columns, axis=1, inplace=True) # Чистим лишние колонки если появятся col_x и col_y
    output.drop_duplicates(inplace=True)
    return output

In [12]:
# Функция объединения df по всем колонкам с использованием merge_column()

def all_bounded_data(big_data, investigation):
    old_length = investigation.shape[0] # Запишем размер начального df
    added_data = 1 # Поставим счетчик для прогона первого цикла
    
    # Выполняем цикл while пока добавляются новые данные по любой из колонок
    while added_data > 0:
        
        # Пройдем первый раз по всем колонкам        
        for column in ['id', 'phone', 'mail']:
            investigation = merge_column(big_data, investigation, column) # Данные добавятся при совпадении по любой колонке
            investigation.drop_duplicates(inplace=True) 
                
        added_data = investigation.shape[0] - old_length # Посчитаем кол-во добавленных строчек. Если 0 - цикл остановится, в другом случае будет выполнен следующий прогон по всем колонкам
        old_length = investigation.shape[0] # Перезадаем длину предыдущей таблицы для подсчета в след цикле
    
    return investigation # После всех циклов возвращается финальный результат со всеми связанными данными

In [13]:
awesome_result_big_data = all_bounded_data(big_data, big_investigation)

write_data(awesome_result_big_data)

         key     id        phone            mail
0          1  57819  87778884800  szOQ@gmail.com
3       7132  46896  87778882733  szOQ@gmail.com
6       7398  23163  87778880400  szOQ@gmail.com
9          2  83851  87778885320  zSbY@gmail.com
12      5589  70519  87778881505  zSbY@gmail.com
...      ...    ...          ...             ...
17380   9977  26249  87778880713  LPZJ@gmail.com
17381   9985  20785  87778880771  NhIQ@gmail.com
17382   9995  21891  87778881737  jljY@gmail.com
17383   9997  72594  87778881808  tvCo@gmail.com
17384  10000  12639  87778880365  unMq@gmail.com

[8185 rows x 4 columns]


# Проверка по условиям задания

In [14]:
# Проверка на данных из задания

small_data = {'key': [1, 2, 3, 4, 5, 6],
      'id': [12345, 54321, 98765, 66678, 34567, 34567],
      'phone': [89997776655, 87778885566, 87776664577,
               87778885566, 87778885566, 89087545678],
      'mail': ['test@mail.ru', 'two@mail.ru', 'three@mail',
               'four@mail.ru', 'four@mail.ru', 'five@mail.ru']}
small_data = pd.DataFrame(data=small_data, index=[1, 2, 3, 4, 5, 6])

In [15]:
small_investigation = data_to_search(small_data)
write_data(small_investigation)

Введите запрос формата <поле> = <значение>. Пример: phone = 87778885566

Что ищем? phone = 87778885566

Для поля phone по запросу 87778885566 найдено 3 записей
   key     id        phone          mail
2    2  54321  87778885566   two@mail.ru
4    4  66678  87778885566  four@mail.ru
5    5  34567  87778885566  four@mail.ru


In [16]:
# Например: если задать поиск по условию phone = 87778885566;

# Для small_data результат должен быть следующим:
# 2;54321;87778885566; two@mail.ru
# 4; 66678;87778885566; four@mail.ru
# 5; 34567; 84547895566; four@mail.ru
# 6; 34567; 89087545678; five@mail.ru

awesome_result_small_data = all_bounded_data(small_data, small_investigation)

write_data(awesome_result_small_data)

   key     id        phone          mail
0    2  54321  87778885566   two@mail.ru
1    4  66678  87778885566  four@mail.ru
3    5  34567  87778885566  four@mail.ru
5    6  34567  89087545678  five@mail.ru
